### Universidad del Valle de Guatemala
### Modelado y Simulacion
#### MICRO PROYECTO 4
##### Juan Marroquin 19845
##### Carlos Raxtum 19721

import libraries

In [1]:
import numpy as np
import math
import random
import matplotlib.pyplot as plt

#### Seed para generador de númerosa aleatorios uniformes

In [2]:
random.seed(69)

#### Generador de Numeros Aleatorios

In [3]:
def Exponential(Lambda):
    return -float((1./Lambda)*math.log(random.random()))

In [4]:
def Poisson(s, lamda):
    return s-(1/lamda)*math.log(random.random())

#### MountainMegaComputing simulator

In [5]:
def MountainMegaComputing(num_sol=2400.,tiempo_benchMark=3600):
    inf = float('inf')
    solicitudesXsegundo = num_sol/60
    t,num_A,num_D,t_arrival,t_departures,n,time_process = 0,0,0,0,0,0,0
    T = tiempo_benchMark # final time
    arrivals,departures = [],[]
    while t<=T:
        if(t_arrival<=t_departures and t_arrival<=T):
            t = t_arrival
            num_A = num_A + 1
            n = n + 1
            t_arrival = Poisson(t,solicitudesXsegundo)
            if n==1:
                Y = Exponential(solicitudesXsegundo)
                t_departures = t + Y
            arrivals.append(t)
        elif(t_departures<t_arrival and t_departures<=T):
            t = t_departures
            n = n - 1
            num_D = num_D +1
            if n == 0:
                t_departures = inf
            else:
                Y = Exponential(solicitudesXsegundo)
                t_departures = t + Y
                t = t_departures
            departures.append(t)
        
        elif(min(t_arrival,t_departures) > T and n > 0):
            t =t_departures
            n = n - 1
            num_D = num_D + 1
            if n>0:
                Y = Exponential(solicitudesXsegundo)
                t_departures = t + Y
                t = t_departures
            departures.append(t)
        elif(min(t_arrival,t_departures) >T and n == 0):
            time_process = max(t - T,0)
            
            break
    
    total = 0
    for k in range(len(departures)):
        total += departures[k]-arrivals[k]
    prom = total/len(arrivals)
    print("ESTE ES EL RESULTADO PARA EL SERVIDOR Mountain Mega Computing")
    print("El numero de llegadas al servidor fue de: ", num_A)
    print("El tiempo que estuvo ocupado el servidor fue: ", round(t_departures,2))
    print("El tiempo que estuvo libre el servidor fue de: ", round(t_departures - t_arrival,2))
    print("El tiempo en total estuvieron las solicitudes en cola: ",round(total,2))
    print("El tiempo promedio que estuvo cada solitud en cola: ",round(prom ,2))
    print("El numero de solicitudes que estuvieron en cola cada segundo: ", round(len(arrivals)/60,2))
    print("El numero de solicitudes en cola al momento de terminar la simulacion: ",round(len(arrivals)-len(departures),2))

#### PizzaComputing simulator

In [36]:
def PizzaComputing(num_server,num_sol=2400.,tiempo_benchMark=3600,show=True):
    inf = float('inf')                             
    T = tiempo_benchMark                                                         
    num_A,t = 0,0
    arrivals,departures,server = [],[],[]                                           
    solicitudesXsegundo = ((num_sol*0.10)/60)
    t_arrival = Exponential(solicitudesXsegundo)             
    t_departures = [100000]*num_server                   
    process_servers = [0]*num_server                        
    solicitude_Server = [0]*(1+num_server)                    
    
    while t < T or solicitude_Server[0] > 0:           
        m = min(t_departures)
        min_time = t_departures.index(m)            
        
        if t_arrival < t_departures[min_time] and t_arrival< T:   
            t = t_arrival                      
            num_A = num_A +  1                
            t_arrival = t_arrival + Exponential(solicitudesXsegundo)
            arrivals.append(t)                   
            
            if solicitude_Server[0] == 0:              
                server.append(t)            
                solicitude_Server[0] += 1              
                solicitude_Server[1] = num_A              
                t_departures[0] = t + Exponential(10) 
                
            elif solicitude_Server[0] < num_server:            
                busy = solicitude_Server.index(0) - 1 
                server.append(t)             
                solicitude_Server[0] += 1              
                solicitude_Server[busy+1] = num_A      
                t_departures[busy] = t + Exponential(solicitudesXsegundo) 
            
            else:                      
                solicitude_Server[0] += 1              
            
        else:                           
            t = t_departures[min_time]              
            process_servers[min_time] += 1             
            departures.append(t)                 
            
            if solicitude_Server[0] <= num_server:            
                solicitude_Server[0] -= 1              
                t_departures[min_time] = 100000     
                solicitude_Server[min_time + 1] = 0      
            
            else:                       
                solicitude_Server[0] *= -1
                next_evente = max(solicitude_Server)+1   
                solicitude_Server[0] *= -1
                server.append(t)                             
                solicitude_Server[0] -= 1              
                t_departures[min_time] = t + Exponential(solicitudesXsegundo) 
                solicitude_Server[min_time + 1] = next_evente 
    total=0
    for k in range(len(arrivals)):
        resta=server[k]-arrivals[k]
        total=total+resta
    prom = total/len(arrivals)
    if show is True:
        print("Solicitudes atendididas por cada servidor:")
        print("-----------------------------------------")
        for i in range (len(process_servers)):
            print("\t"+"Servidor "+str(i+1)+":", process_servers[i], "solicitudes.")
        print("-----------------------------------------")
        print("Tiempo total: ", sum(t_departures), "segundos.")
        print("Solicitudes atendidas: ", num_A)
        print("Ultima llegada:", round(arrivals[-1],2), "s.")
        print("Ultima salida:", round(departures[-1],2), "s.")
        print("-----------------------------------------\n")
    print("ESTE ES EL RESULTADO PARA EL SERVIDOR Pizza Computing")
    print("El numero de llegadas al servidor fue de: ", num_A)
    print("El tiempo que estuvo ocupado el servidor fue: ", round(sum(t_departures),2))
    print("El tiempo que estuvo libre el servidor fue de: ", round(sum(t_departures) - t_arrival,2))
    print("El tiempo en total estuvieron las solicitudes en cola: ",round(total,2))
    print("El tiempo promedio que estuvo cada solitud en cola: ",round(prom ,2))
    print("El numero de solicitudes que estuvieron en cola cada segundo: ", round(len(arrivals)/60,2))
    print("El numero de solicitudes en cola al momento de terminar la simulacion: ",round(len(arrivals)-len(departures),2))

#### Task 1
#### Modele, simule y analice el comportamiento de ambos sistemas durante una hora de ejecución

#### Mountain Mega Computing

In [37]:
MountainMegaComputing()

ESTE ES EL RESULTADO PARA EL SERVIDOR Mountain Mega Computing
El numero de llegadas al servidor fue de:  144309
El tiempo que estuvo ocupado el servidor fue:  3600.02
El tiempo que estuvo libre el servidor fue de:  0.02
El tiempo en total estuvieron las solicitudes en cola:  1139793.75
El tiempo promedio que estuvo cada solitud en cola:  7.9
El numero de solicitudes que estuvieron en cola cada segundo:  2405.15
El numero de solicitudes en cola al momento de terminar la simulacion:  316


#### Pizza Computing

In [38]:
PizzaComputing(6,show=True)

Solicitudes atendididas por cada servidor:
-----------------------------------------
	Servidor 1: 9580 solicitudes.
	Servidor 2: 3172 solicitudes.
	Servidor 3: 1264 solicitudes.
	Servidor 4: 394 solicitudes.
	Servidor 5: 107 solicitudes.
	Servidor 6: 22 solicitudes.
-----------------------------------------
Tiempo total:  600000 segundos.
Solicitudes atendidas:  14539
Ultima llegada: 3599.99 s.
Ultima salida: 3600.07 s.
-----------------------------------------

ESTE ES EL RESULTADO PARA EL SERVIDOR Pizza Computing
El numero de llegadas al servidor fue de:  14539
El tiempo que estuvo ocupado el servidor fue:  600000
El tiempo que estuvo libre el servidor fue de:  596399.32
El tiempo en total estuvieron las solicitudes en cola:  0.6
El tiempo promedio que estuvo cada solitud en cola:  0.0
El numero de solicitudes que estuvieron en cola cada segundo:  242.32
El numero de solicitudes en cola al momento de terminar la simulacion:  0


#### Task 2
#### Determine empíricamente cuántos servidores se necesitaría “alquilar” en Pizzita computing para asegurar que siempre habrá al menos un servidor disponible para atender una solicitud dada

In [39]:
for i in range(1,16):
    print("-----------------------------------------")
    print('PARA ',i,' SERVIDORES')
    PizzaComputing(i,show=False)
    print("-----------------------------------------\n")

-----------------------------------------
PARA  1  SERVIDORES
ESTE ES EL RESULTADO PARA EL SERVIDOR Pizza Computing
El numero de llegadas al servidor fue de:  14510
El tiempo que estuvo ocupado el servidor fue:  100000
El tiempo que estuvo libre el servidor fue de:  96399.94
El tiempo en total estuvieron las solicitudes en cola:  291613.89
El tiempo promedio que estuvo cada solitud en cola:  20.1
El numero de solicitudes que estuvieron en cola cada segundo:  241.83
El numero de solicitudes en cola al momento de terminar la simulacion:  0
-----------------------------------------

-----------------------------------------
PARA  2  SERVIDORES
ESTE ES EL RESULTADO PARA EL SERVIDOR Pizza Computing
El numero de llegadas al servidor fue de:  14307
El tiempo que estuvo ocupado el servidor fue:  200000
El tiempo que estuvo libre el servidor fue de:  196399.95
El tiempo en total estuvieron las solicitudes en cola:  634.54
El tiempo promedio que estuvo cada solitud en cola:  0.04
El numero de so

#### Task 3
#### Se espera que apartir del tercer año del lanzamiento de su aplicación , la cantidad de usuarios sufra un alza, y por tanto deberán atender como máximo 6000 solicitudes por minuto

#### Mountain Mega Computing

In [40]:
MountainMegaComputing(num_sol=6000)

ESTE ES EL RESULTADO PARA EL SERVIDOR Mountain Mega Computing
El numero de llegadas al servidor fue de:  359913
El tiempo que estuvo ocupado el servidor fue:  3600.01
El tiempo que estuvo libre el servidor fue de:  0.03
El tiempo en total estuvieron las solicitudes en cola:  1483381.68
El tiempo promedio que estuvo cada solitud en cola:  4.12
El numero de solicitudes que estuvieron en cola cada segundo:  5998.55
El numero de solicitudes en cola al momento de terminar la simulacion:  1109


#### Pizza Computing

In [43]:
PizzaComputing(8,show=True,num_sol=6000)

Solicitudes atendididas por cada servidor:
-----------------------------------------
	Servidor 1: 18081 solicitudes.
	Servidor 2: 10877 solicitudes.
	Servidor 3: 4824 solicitudes.
	Servidor 4: 1595 solicitudes.
	Servidor 5: 375 solicitudes.
	Servidor 6: 75 solicitudes.
	Servidor 7: 12 solicitudes.
	Servidor 8: 2 solicitudes.
-----------------------------------------
Tiempo total:  800000 segundos.
Solicitudes atendidas:  35841
Ultima llegada: 3599.95 s.
Ultima salida: 3600.05 s.
-----------------------------------------

ESTE ES EL RESULTADO PARA EL SERVIDOR Pizza Computing
El numero de llegadas al servidor fue de:  35841
El tiempo que estuvo ocupado el servidor fue:  800000
El tiempo que estuvo libre el servidor fue de:  796399.94
El tiempo en total estuvieron las solicitudes en cola:  0.0
El tiempo promedio que estuvo cada solitud en cola:  0.0
El numero de solicitudes que estuvieron en cola cada segundo:  597.35
El numero de solicitudes en cola al momento de terminar la simulacion: 

#### Task 4
#### Emita una recomendación para la junta directiva